# PLN - Analisis del Sentimiento

Vamos a analizar el sentimiento en las letras de canciones, para ello necesitamos extraer las letras de las canciones de una base de datos en la red. Importamos las bibliotecas para acceder a direcciones url y Beautiful Soup para extraer informacion especifica de estas paginas web.

In [ ]:
import urllib
# Si tu entorno no tiene Beautiful Soup instalado:
#!pip install BeautifulSoup4
from bs4 import BeautifulSoup

Seleccionamos una cancion en concreto, abrimos la pagina y lo transformamos en un objeto de Beautiful Soup:

In [ ]:
url = "https://www.mldb.org/song-54050-my-friend-of-misery.html"
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html)

Buscamos la referencia que necesitamos, en esta pagina, la clase "songtext":

In [ ]:
for p in soup.findAll("p", {"class": "songtext"}):
  cancion = p.get_text() 
  print(cancion)

My Friend Of Misery

You just stood there screaming
fearing no one was listening to you
they say the empty can rattles the most
the sound of your voice must soothe you
hearing only what you want to hear
and knowing only what you've heard
you you're smothered in tragedy
you're out to save the world 

misery
you insist that the weight of the world
should be on your shoulders
misery
there's much more to life than what you see
my friend of misery

you still stood there screaming
no one caring about these words you tell
my friend before your voice is gone
one man's fun is another's hell
these times are sent to try men's souls
but something's wrong with all you see
you you'll take it on all yourself
remember, misery loves company 

misery
you insist that the weight of the world 
should be on your shoulders
misery
there's much more to life than what you see
my friend of misery

you just stood there creaming 
my friend of misery


Vamos a utilizar un modelo pre-entrenado para la deteccion de sentimiento en textos:

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

In [ ]:
sentimiento = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


In [ ]:
pos_text = 'the sound of your voice must soothe you'
neg_text = 'my friend of misery'

In [ ]:
resultado = sentimiento(pos_text)[0]
print("Etiqueta:", resultado['label'])
print("Confianza:", resultado['score'])

Etiqueta: POSITIVE
Confianza: 0.9992879033088684


In [ ]:
resultado = sentimiento(neg_text)[0]
print("Etiqueta:", resultado['label'])
print("Confianza:", resultado['score'])

Etiqueta: NEGATIVE
Confianza: 0.9902848601341248


In [ ]:
resultado = sentimiento(cancion)[0]
print("Etiqueta:", resultado['label'])
print("Confianza:", resultado['score'])

Etiqueta: NEGATIVE
Confianza: 0.9941691160202026


Podemos utilizar otros modelos, entrenados en otros idiomas:

In [ ]:
url = "https://www.mldb.org/song-66272-celia-cruz-la-vida-es-un-carnaval.html"
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html)
for p in soup.findAll("p", {"class": "songtext"}):
  cancion = p.get_text()[:2+len(cancion)//2]
  print(cancion)

Todo aquel que piense que la vida es desigual,
tiene que saber que no es así,
que la vida es una hermosura, hay que vivirla.
Todo aquel que piense que está solo y que está mal,
tiene que saber que no es así,
que en la vida no hay nadie solo, siempre hay alguien.

Ay, no ha que llorar, que la vida es un carnaval,
es más bello vivir cantando.
Oh, oh, oh, Ay, no hay que llorar,
que la vida es un carnaval
y las penas se van cantando.

Todo aquel que piense que la vida 


Podemos utilizar [pysentimiento](https://github.com/pysentimiento) para analizar el sentimiendo en español.

In [ ]:
# Con %% capture podemos evitar que el cuaderno
# muestre el output de una celda de instalacion por ejemplo,
# cuya salida puede ser no muy informativa.
%%capture
!pip install pysentimiento

Creamos un objecto analizador de sentimiento:

In [ ]:
from pysentimiento import create_analyzer
analizador = create_analyzer(task="sentiment", lang="es")

Downloading:   0%|          | 0.00/334 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/838k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/925 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/415M [00:00<?, ?B/s]

Predecimos que sentimiento pinesa que transmite la cancion:

In [ ]:
analizador.predict(cancion)

AnalyzerOutput(output=NEU, probas={NEU: 0.868, POS: 0.128, NEG: 0.004})

Podemos acceder tanto al resultado como a las probabilidades:

In [ ]:
analizador.predict(cancion).output

'NEU'

In [ ]:
analizador.predict(cancion).probas

{'NEG': 0.0038446919061243534,
 'NEU': 0.8683628439903259,
 'POS': 0.12779250741004944}

Le puede costar al modelo diferenciar entre neutralidad y positividad, si vemos cada linea de una cancion, podemos ver donde puede cometer errores de contexto:

In [ ]:
for linea in cancion.split("\n"):
  print(f'{linea}: {analizador.predict(linea).output}')

Todo aquel que piense que la vida es desigual,: NEU
tiene que saber que no es así,: NEG
que la vida es una hermosura, hay que vivirla.: POS
Todo aquel que piense que está solo y que está mal,: NEG
tiene que saber que no es así,: NEG
que en la vida no hay nadie solo, siempre hay alguien.: POS
: NEU
Ay, no ha que llorar, que la vida es un carnaval,: NEU
es más bello vivir cantando.: POS
Oh, oh, oh, Ay, no hay que llorar,: NEU
que la vida es un carnaval: NEU
y las penas se van cantando.: NEU
: NEU
Todo aquel que piense que la vida : NEU


# Ejercicio
Como ejercicio, utilizando los ejemplos de arriba, y de la manera mas automatica posible, analiza la discografia de un artista o grupo musical de tu eleccion. ¿Son sus letras positivas o negativas en general? ¿Donde puede estar comentiendo errores el modelo de prediccion?

In [ ]:
# Tu codigo aqui: